In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
import os

/home/phiha/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_features = pd.read_csv(os.path.join('data', 'NUSW-NB15_features.csv'), header=None).values
features = data_features[1:, :]
feature_names = features[:, 1]  # Names of the features in a list
feature_types = np.array(
    [item.lower() for item in features[:-2, 2]])  # The types of the corresponding features in 'features_names'

In [3]:
nominal_cols = np.where(feature_types == "nominal")[0]
integer_cols = np.where(feature_types == "integer")[0]
float_cols = np.where(feature_types == "float")[0]
binary_cols = np.where(feature_types == "binary")[0]

In [4]:
dataframe1 = pd.read_csv(os.path.join("data","UNSW-NB15_1.csv"), header=None)
dataframe2 = pd.read_csv(os.path.join("data","UNSW-NB15_2.csv"), header=None)
dataframe3 = pd.read_csv(os.path.join("data","UNSW-NB15_3.csv"), header=None)
dataframe4 = pd.read_csv(os.path.join("data","UNSW-NB15_4.csv"), header=None)

/home/phiha/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/phiha/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print('Concatenating...')
dataframe = pd.concat([dataframe1, dataframe2, dataframe3, dataframe4])

del dataframe1
del dataframe2
del dataframe3
del dataframe4

Concatenating...


In [ ]:
# dataframe.columns = feature_names

In [ ]:
#dataframe.service.value_counts()

In [ ]:
dataframe.columns[47]

In [ ]:
dataframe.iloc[:,13].unique()

In [6]:
dataframe[47].fillna("normal", inplace=True)


In [ ]:
df2 = dataframe.iloc[-10:, 10:20]

In [ ]:
df2

In [7]:
attact_cat = dataframe.iloc[:, 47]
labels = dataframe.iloc[:, 48]
labels.fillna(0, inplace=True)
dataframe = dataframe.iloc[:, :-2]

In [8]:
my_dict = {" Fuzzers": "Fuzzers", ' Fuzzers ': 'Fuzzers',
          'Backdoor' : 'Backdoors', ' Reconnaissance ': 'Reconnaissance', ' Shellcode ' : 'Shellcode'}
attact_cat.replace(my_dict, inplace=True)
attact_cat.unique()
# ' Fuzzers', ' Fuzzers ', ' Reconnaissance ', ' Shellcode ',
#         'Analysis', 'Backdoor', 'Backdoors', 'DoS', 'Exploits', 'Generic',
#         'Reconnaissance', 'Shellcode', 'Worms', 'normal']

array(['normal', 'Exploits', 'Reconnaissance', 'DoS', 'Generic',
       'Shellcode', 'Fuzzers', 'Worms', 'Backdoors', 'Analysis'], dtype=object)

In [9]:
df = pd.get_dummies(dataframe, columns=nominal_cols[2:])
df.replace(r'\s+', np.nan, regex=True, inplace=True)
df.drop(df.columns[[0,1,2,3]], axis=1, inplace=True)
df.fillna(0, inplace=True)

In [ ]:
df.iloc[-10:,191:]

In [12]:
df1 = df.iloc[-10:,191:]

In [11]:
import re
df1.rename(columns=lambda x: re.sub('13','srv',x))

,srv_-,srv_dhcp,srv_dns,srv_ftp,srv_ftp-data,srv_http,srv_irc,srv_pop3,srv_radius,srv_smtp,srv_snmp,srv_ssh,srv_ssl
440034,1,0,0,0,0,0,0,0,0,0,0,0,0
440035,1,0,0,0,0,0,0,0,0,0,0,0,0
440036,0,0,0,0,0,1,0,0,0,0,0,0,0
440037,0,0,0,1,0,0,0,0,0,0,0,0,0
440038,1,0,0,0,0,0,0,0,0,0,0,0,0
440039,0,0,0,0,1,0,0,0,0,0,0,0,0
440040,0,0,0,1,0,0,0,0,0,0,0,0,0
440041,0,0,0,1,0,0,0,0,0,0,0,0,0
440042,0,0,0,0,0,1,0,0,0,0,0,0,0
440043,0,0,0,0,0,0,0,1,0,0,0,0,0


In [26]:
import time

In [13]:
X = df.astype(np.float64)

In [14]:
X.shape

(2086976, 204)

In [15]:
y = labels.astype(np.float64)

In [16]:
y.shape

(2086976,)

In [15]:
seed = 125
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)
# fit model no training data


NameError: name 'X' is not defined

In [ ]:
#BAGGING TREE
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier()
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
print confusion_matrix(y_true=y_test, y_pred=y_pred)

In [ ]:
#RF
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
print confusion_matrix(y_true=y_test, y_pred=y_pred)

In [ ]:
model = XGBClassifier()
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print confusion_matrix(y_true=y_test, y_pred=y_pred)

In [28]:
from sklearn.metrics import classification_report


In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.model_selection import trai

In [18]:
y = attact_cat.values
seed = 125
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)

In [19]:
np.unique(y_train, return_counts=True)

(array(['Analysis', 'Backdoors', 'DoS', 'Exploits', 'Fuzzers', 'Generic',
        'Reconnaissance', 'Shellcode', 'Worms', 'normal'], dtype=object),
 array([   2142,    1863,   13082,   35620,   19397,  172385,   11189,
           1209,     139, 1412554]))

In [20]:
np.unique(y_test, return_counts=True)

(array(['Analysis', 'Backdoors', 'DoS', 'Exploits', 'Fuzzers', 'Generic',
        'Reconnaissance', 'Shellcode', 'Worms', 'normal'], dtype=object),
 array([   535,    466,   3271,   8905,   4849,  43096,   2798,    302,
            35, 353139]))

In [ ]:
#BAGGING TREE
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(verbose=True, n_jobs=-1)
start = time.time()
model.fit(X_train, y_train)
print model
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
#print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

In [ ]:
#RF
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(verbose=True, n_jobs=-1)
print model
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
#print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

In [ ]:
#RF
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
print model
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
#print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

In [36]:
#XGBoost
# model = XGBClassifier(n_estimators=100, max_depth=6)
# start = time.time()
# model.fit(X_train, y_train)
# print model
# end = time.time()
# print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

[[    20      0     65    340      1      2      0      0      0    107]
 [     0     47     65    334      5      1      3      4      0      7]
 [     0      0    665   2451     26     28     18     25      0     58]
 [     0      1    536   7818     79     64    122     29      4    252]
 [     0      0     78    511   2335      7      8     18      0   1892]
 [     5      2     94    517     31  42379      2     10      2     54]
 [     0      0     90    552      7      2   2136      2      0      9]
 [     0      0      2     37     14      2     15    185      0     47]
 [     0      0      0      5      1      2      1      0     25      1]
 [     1      2      3    102    737      9      6     25      0 352254]]
                precision    recall  f1-score   support

      Analysis       0.04      0.77      0.07        26
     Backdoors       0.10      0.90      0.18        52
           DoS       0.20      0.42      0.27      1598
      Exploits       0.88      0.62      0.7

In [34]:
#XGBoost
model1 = XGBClassifier()
start = time.time()
model1.fit(X_train, y_train)
print model
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model1.predict(X_test)
print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
Training in 499.17
[[    12      0      1    413      1      0      2      0      0    106]
 [     0     16      2    413      5      1      6      0      0     23]
 [     0      0     40   3051     26     11     36     17      0     90]
 [     0      0     30   8191    108     34    158     14      1    369]
 [     1      0      0    653   1895      5     17      9      0   2269]
 [     5      0      5    791     50  42126      8      4      0    107]
 [     0      0      2    643     12      2   2116      2      0     21]
 [     0      0      0     71     16      1     42    102      0     70]
 [     0      0      0     18      2      1      0      0     11  

In [17]:
from sklearn.model_selection import GridSearchCV

In [21]:
_, X_sample, _, y_sample = train_test_split(X_train, y_train, test_size=0.1, random_state=452, stratify=y_train)

In [ ]:
#grid search
alg = XGBClassifier()
clf = GridSearchCV(alg,{'max_depth': [3,6,9,12],
                    'n_estimators': [10, 20, 50,100,200],
                       }, 
                    verbose=2, 
                    scoring='f1_macro', n_jobs = -1)

clf.fit(X_sample,y_sample)
clf.best_score_, clf.best_params_

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] n_estimators=10, max_depth=3 ....................................
[CV] n_estimators=10, max_depth=3 ....................................
[CV] n_estimators=10, max_depth=3 ....................................
[CV] n_estimators=20, max_depth=3 ....................................
[CV] n_estimators=20, max_depth=3 ....................................
[CV] n_estimators=20, max_depth=3 ....................................
[CV] n_estimators=50, max_depth=3 ....................................
[CV] n_estimators=50, max_depth=3 ....................................
[CV] n_estimators=50, max_depth=3 ....................................
[CV] n_estimators=100, max_depth=3 ...................................
[CV] n_estimators=100, max_depth=3 ...................................
[CV] n_estimators=100, max_depth=3 ...................................
[CV] n_estimators=200, max_depth=3 ...................................
[CV] n_estimator

/home/phiha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] .................... n_estimators=10, max_depth=12, total= 2.0min


/home/phiha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/phiha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] .................... n_estimators=10, max_depth=12, total= 2.1min
[CV] .................... n_estimators=10, max_depth=12, total= 2.1min


/home/phiha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] .................... n_estimators=20, max_depth=12, total= 3.7min


/home/phiha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/phiha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] .................... n_estimators=20, max_depth=12, total= 3.8min
[CV] .................... n_estimators=20, max_depth=12, total= 3.8min


In [23]:
cl

(0.97503563770529111, {'max_depth': 6, 'n_estimators': 100})

In [ ]:
# start = time.time()
# seed = 125
# test_size = 0.3
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model no training data
# model = XGBClassifier()
# model.fit(X_train, y_train)
# end = time.time()
# print 'Training in %.2f'%(end-start)

In [24]:
y_pred = clf.predict(X_test)

XGBoostError: need to call fit beforehand

In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
print(classification_report(y_pred,y_test))